# 기본 Chatbot을 이용해서 Langraph 구현하기

StateGraph : 기본 챗봇의 구조를 "상태 머신"으로 정의한다. 
Nodes : 우리의 챗봇을 호출할 수 있는 LLM과 함수를 작성하기위한 Node를 추가 
Edge : 함수들 사이를 봇이 어떻게 전환할 것인지를 특정하기 위한 Edge를 추가 

## 라이브러리 설치

In [3]:
%%capture

# update or install the necessary libraries

%pip install langgraph

In [ ]:
import openai
import os
import IPython
from dotenv import load_dotenv

load_dotenv()

## 기본 코드 작성

### StateGraph 정의 

In [4]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

### Node / Edge 정의 

Node들은 작업의 단위(Unit Of Work)를 나타낸다. 

In [6]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # or whichever OpenAI model you prefer
    temperature=0.7             # optional: adjust as needed
)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_name': 'gpt-3.5-t...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

### Graph 확인하기 

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass